Define a function fetch_stock_info which takes a list of stocks, fetches yahoo finance ticker stock info for each stock, and returns a pandas Dataframe populated with all fundamental indicators.

In [1]:
import yfinance
import pandas
import numpy
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

def fetch_stock_info(stocks):

    key_yfinancekey_map = {
        'Symbol': 'symbol',
        'Name': 'shortName',
        'Industry': 'industry',
        'Forward Earnings Per Share (EPS)': 'forwardEps',
        'Forward Price to Earnings Ratio (P/E)': 'forwardPE',
        'Price/Earnings-to-Growth Ratio (PEG)': 'pegRatio',
        'Price to Book Ratio (P/B)': 'priceToBook',
        'Return on Equity (ROE)': 'returnOnEquity',
        '12-Month Trailing Price-to-Sales Ratio (P/S)': 'priceToSalesTrailing12Months',
        'Dividend Payout Ratio (DPR)': 'payoutRatio',
        'Dividend Yield (DY)': 'dividendYield',
        'Current Ratio (CR)': 'currentRatio',
        'Free Cash Flow Yield (FCFY)': 'freeCashflow',
        'Beta': 'beta',
        'Price': 'currentPrice',
        '52-Week Low': 'fiftyTwoWeekLow',
        '52-Week High': 'fiftyTwoWeekHigh'
    }

    key_list = key_yfinancekey_map.keys()
    stock_data = {}

    for ticker in stocks:
        # GET Yahoo Finance TICKER INFO
        yfTicker = yfinance.Ticker(ticker)
        stock_info = yfTicker.info

        for stock_data_key in key_list:
            try:
                stock_data[stock_data_key].append(stock_info[key_yfinancekey_map[stock_data_key]])
            except KeyError:
                if stock_data_key not in stock_data:  # First list entry
                    stock_data[stock_data_key] = []
                    stock_data[stock_data_key].append(stock_info[key_yfinancekey_map[stock_data_key]])
                else:   # Couldn't fetch data for this stat for this ticker
                    stock_data[stock_data_key].append(numpy.nan)

    # Return a DF using the stock_data dictionary
    return pandas.DataFrame(stock_data)

Define a styling function for our fundamental analysis dataframe

In [2]:
def style_fundamentals_dataframe(styler):
    # Column formatting
    styler.format({'Forward Earnings Per Share (EPS)': '${:.2f}', 'Forward Price to Earnings Ratio (P/E)': '{:.2f}',
                   'Price/Earnings-to-Growth Ratio (PEG)': '{:.2f}',
                   'Free Cash Flow Yield (FCFY)': '{:.2f}', 'Price to Book Ratio (P/B)': '{:.2f}', 'Return on Equity (ROE)': '{:.2f}',
                   '12-Month Trailing Price-to-Sales Ratio (P/S)': '{:.2f}',
                   'Dividend Payout Ratio (DPR)': '{:.2f}', 'Dividend Yield (DY)': '{:.2f}%',
                   'Current Ratio (CR)': '{:.2f}', 'Beta': '{:.2f}', '52-Week Low': '${:.2f}',
                   'Price': '${:.2f}', '52-Week High': '${:.2f}'})

    # Grid
    styler.set_properties(**{'border': '0.1px solid black'})

    # Set background gradients
    styler.background_gradient(subset=['Forward Earnings Per Share (EPS)'], cmap='Greens')
    styler.background_gradient(subset=['Forward Price to Earnings Ratio (P/E)'], cmap='Greens')
    styler.background_gradient(subset=['Price/Earnings-to-Growth Ratio (PEG)'], cmap='Greens')
    styler.background_gradient(subset=['Price to Book Ratio (P/B)'], cmap='Greens')
    styler.background_gradient(subset=['Return on Equity (ROE)'], cmap='Greens')
    styler.background_gradient(subset=['12-Month Trailing Price-to-Sales Ratio (P/S)'], cmap='Greens')
    styler.background_gradient(subset=['Dividend Payout Ratio (DPR)'], cmap='Greens')
    styler.background_gradient(subset=['Dividend Yield (DY)'], cmap='Greens')
    styler.background_gradient(subset=['Current Ratio (CR)'], cmap='Greens')

    # Remove index column
    styler.hide(axis='index')

    # Left justify some columns
    styler.set_properties(subset=['Symbol', 'Name', 'Industry'], **{'text-align': 'left'})
    return styler

Create a list of stock tickers to represent the asset basket, perform fundamental analysis, style the dataframe for presentation

In [3]:
stocks = ['AAPL', 'GS', 'IBM', 'INTC', 'JNJ', 'JPM', 'MS', 'TRV', 'GOOG', 'TSLA']

# Perform fundamental analysis on the companies
stock_info_dataframe = fetch_stock_info(stocks)

# Style result Dataframe
stock_info_dataframe.style.pipe(style_fundamentals_dataframe).set_caption('Fundamental Analysis Indicators').set_table_styles(
    [{
        'selector': 'th.col_heading',
        'props': 'text-align: center'
    }, {
        'selector': 'caption',
        'props': [('text-align', 'center'),
                  ('font-size', '14pt'),
                  ('font-weight', 'bold')]}
    ])

Symbol,Name,Industry,Forward Earnings Per Share (EPS),Forward Price to Earnings Ratio (P/E),Price/Earnings-to-Growth Ratio (PEG),Price to Book Ratio (P/B),Return on Equity (ROE),12-Month Trailing Price-to-Sales Ratio (P/S),Dividend Payout Ratio (DPR),Dividend Yield (DY),Current Ratio (CR),Free Cash Flow Yield (FCFY),Beta,Price,52-Week Low,52-Week High
AAPL,Apple Inc.,Consumer Electronics,$7.13,23.74,2.36,35.32,1.54,6.78,0.15,0.01%,1.07,86563127296.00,1.28,$169.30,$164.08,$199.62
GS,"Goldman Sachs Group, Inc. (The)",Capital Markets,$40.92,10.45,0.61,1.34,0.08,2.97,0.42,0.03%,2.81,nan,1.42,$427.57,$289.36,$428.53
IBM,International Business Machines,Information Technology Services,$10.55,15.84,5.27,6.59,0.36,2.47,0.75,0.04%,1.13,10952250368.00,0.71,$167.13,$120.55,$199.18
INTC,Intel Corporation,Semiconductors,$2.22,14.36,0.61,1.28,0.04,2.46,1.85,0.02%,1.57,-14835125248.00,1.01,$31.88,$26.86,$51.28
JNJ,Johnson & Johnson,Drug Manufacturers - General,$11.00,13.29,2.63,5.11,nan,4.11,0.64,0.03%,nan,nan,0.53,$146.14,$143.13,$175.97
JPM,JP Morgan Chase & Co.,Banks - Diversified,$16.45,11.76,9.98,1.81,0.16,3.71,0.26,0.02%,nan,nan,1.12,$193.49,$131.81,$200.94
MS,Morgan Stanley,Capital Markets,$7.61,12.20,1.68,1.67,nan,2.77,0.60,0.04%,nan,nan,1.41,$92.83,$69.42,$95.57
TRV,"The Travelers Companies, Inc.",Insurance - Property & Casualty,$20.91,10.21,0.74,1.95,0.13,1.14,0.30,0.02%,0.33,12980124672.00,0.59,$213.45,$157.33,$232.75
GOOG,Alphabet Inc.,Internet Content & Information,$7.85,22.13,1.21,7.34,0.30,6.75,nan,nan%,2.15,55064125440.00,1.05,$173.69,$104.50,$176.42
TSLA,"Tesla, Inc.",Auto Manufacturers,$3.43,49.06,4.29,8.34,0.24,5.66,nan,nan%,1.72,-633124992.00,2.44,$168.29,$138.80,$299.29


Key indicators dictionary:

EPS (Earnings Per Share) — 
Portion of a company’s profit that is assigned to each share of its stock

P/E (Price to Earnings) — 
Relationship between the stock price of a company and its per-share earnings. It helps investors determine if a stock is undervalued or overvalued relative to others in the same sector.

PEG (Projected Earnings Growth) — 
Calculated by dividing a stock’s P/E by its projected 12-month forward revenue growth rate. In general, a PEG lower than 1 is a good sign, and a PEG higher than 2 indicates that a stock may be overpriced

FCFY (Free Cash Flow Yield) — 
A financial solvency ratio that compares the free cash flow per share a company is expected to earn against its market value per share. A lower ratio indicates a less attractive investment opportunity.

PB (Price to Book) — 
A ratio of 1 indicates the company’s shares are trading in line with its book value. A P/B higher than 1 suggests the company is trading at a premium to book value, and a P/B lower than 1 indicates a stock that may be undervalued relative to the company’s assets.

ROE (Return on Equity) — 
Provides a way for investors to evaluate how effectively a company is using its equity to generate profits. A higher ROE indicates a more efficient use of shareholder equity, which can lead to increased demand for shares and higher stock price, as well as increase in company’s profits in the future.

P/S (Price to Sales) — 
Determines the fair value of a stock by utilizing a company’s market capitalization and revenue. It shows how much the market values the company’s sales, which can be effective in valuing growth stocks that have yet to turn a profit or aren’t performing as expected due to a temporary setback.

DPR (Dividend Payment Ratio) — 
A ratio of the total amount of dividends paid out to shareholders relative to the net income of the company.

DY (Dividend Yield Ratio) — 
A ratio looks at the amount paid by a company in dividends every year relative to its share price. It is an estimate of the dividend-only return of a stock investment.

CR (Current Ratio) — 
measures a company’s ability to pay off its current liabilities (payable within one year) with its current assets, such as cash, accounts receivable, and inventories. The higher the ratio, the better the company’s liquidity position.

Beta — a measure of a stock’s volatility in relation to the overall market. A stock that swings more than the market over time has a beta above 1.0. If a stock moves less than the market, the stock’s beta is less than 1.0.
